In [1]:
import platform

print(platform.system())

Windows


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import csv
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.layers import BatchNormalization, Dropout, Dense, GlobalAveragePooling2D



In [3]:
def read_csv(train_test):
  lines = []
  #with open(f"gdrive/Shareddrives/IC - Datasets/dataset/lens/CogentAnnotation{train_test}.csv", "r") as file:
  with open(f"lens/CogentAnnotation{train_test}.csv", "r") as file:
      
    csvreader = csv.reader(file)

    for row in csvreader:
        lines.append(np.asarray(row))

  lines.pop(0) #Removing the header of the file
  lines = np.array(lines).T #Transposing to recover the data easier

  return lines

def label_classifier(labels):
    #Changing the string values to numerical using the item value to identify in the dictionary the corresponding item
    type_labels = {
        "Colored": 0,
        "Normal": 1,
        "Transparent": 2
    }

    num_labels = len(labels)
    for i in range (0, num_labels):
        labels[i] = type_labels[labels[i]]

    return labels.astype(np.int32)

In [4]:
def read_images(train_test):

    #Reading csv to get image path transposed to recover easier the data
    csv_info = read_csv(train_test)

    images_path = csv_info[0]
    labels = label_classifier(csv_info[1])
    
    images = []

    for image in images_path:
        img = tf.keras.preprocessing.image.load_img("lens/" + image + ".bmp", target_size=(300,300, 3))
        img_array = keras.preprocessing.image.img_to_array(img)
        images.append(img_array)
      
    
    images = np.array(images)
    images = tf.keras.utils.normalize(images)

    labels = np.array(labels)

    #Shuffle all the data not to keep well separeted
    ts = int(time.time())
    np.random.seed(ts)
    np.random.shuffle(images)
    np.random.seed(ts)
    np.random.shuffle(labels)

    return images, labels
    

In [7]:
x_dataset, y_dataset = read_images("Train")
x_test, y_test = read_images("Test")

In [6]:
class LensModel(tf.keras.Model):
    def __init__(self):
        super(LensModel, self).__init__()

        self.efficientNetB3 = tf.keras.applications.EfficientNetB3(weights='imagenet', include_top=False, pooling=None)
        self.efficientNetB3.trainable = False

        self.linear_layers = tf.keras.Sequential([
            BatchNormalization(),
            Dropout(0.2),
            
            Dense(512),
            tf.keras.layers.ReLU(),
            BatchNormalization(),
            Dropout(0.2),
            
            Dense(512),
            tf.keras.layers.ReLU(),
            BatchNormalization(),
            
            Dense(3),
            tf.keras.layers.Softmax()
        ])

    def call(self, inputs):
        x = self.efficientNetB3(inputs)
        x = tf.keras.layers.Flatten()(x)
        x = self.linear_layers(x)
        return x


model = LensModel()

filepath = "best_model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

optimization = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    optimizer=optimization, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

In [ ]:
# print(x_dataset.shape)
# print(x_test.shape)
# print(y_dataset.shape)
# print(y_dataset[0])

# num_classes = len(np.unique(y_dataset))
# print("Number of classes:", num_classes)

# y_dataset_updated = tf.keras.utils.to_categorical(y_dataset.T, num_classes=2)
# y_test_updated = tf.keras.utils.to_categorical(y_test.T, num_classes=2)

# print(y_test_updated.shape)

In [8]:
epochs = 20 
hist = model.fit(x_dataset, y_dataset, 
                 epochs=epochs, 
                 batch_size=20, 
                 validation_data=(x_test, y_test), 
                 #callbacks=[checkpoint], 
                 verbose = 1
                )

Epoch 1/20
 8/88 [=>............................] - ETA: 5:35 - loss: 2.9655 - accuracy: 0.3750

KeyboardInterrupt: 